In [1]:
# ==== 설치(버전 호환 수정) ====
!pip -q install torch==2.6.0 torchaudio==2.6.0 torchvision==0.21.0 --extra-index-url https://download.pytorch.org/whl/cu121
!pip -q install transformers datasets accelerate sentencepiece --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch, random, numpy as np

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

MODEL_NAME = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to("cuda" if torch.cuda.is_available() else "cpu")

gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)

prompts = [
    "오늘은 날씨가 좋아서",
    "인공지능이 바꿀 미래는",
    "한국 영화 역사에서 가장 인상 깊었던 장면은"
]

outputs = gen(
    prompts,
    max_new_tokens=60,
    do_sample=True,
    top_p=0.9,
    temperature=0.8,
    repetition_penalty=1.2,
    num_return_sequences=1,          # 한 프롬프트당 1개 생성
    return_full_text=True            # 프롬프트 포함해 출력(원하면 False)
)

for i, outs in enumerate(outputs):   # outs 는 리스트
    # 한 프롬프트당 여러 결과일 수 있으므로 다시 loop
    for j, o in enumerate(outs):
        print(f"\n[프롬프트 {i+1}-{j+1}] {prompts[i]}\n=> {o['generated_text']}")


Device set to use cuda:0



[프롬프트 1-1] 오늘은 날씨가 좋아서
=> 오늘은 날씨가 좋아서 그런지 오늘 저녁이 아주 잘 먹었어요."
"그래, 그럼. 뭐든 다 먹어."
나는 녀석이 내 앞에 놓인 맥주를 들이켰다.
그것은 곧 녀석의 입에 들어가기 시작했다.
"너무 맛있어! 그냥 먹어도 된다니까?"
"응,

[프롬프트 2-1] 인공지능이 바꿀 미래는
=> 인공지능이 바꿀 미래는 없다.
우리가 사는 세상도 마찬가지다.
모든 인간은 기계처럼 살고 싶어한다.
하지만 사람들은 로봇을 좋아하지 않는다.
그들은 우리를 위한 기계를 만들 수도 있고, 세상을 변화시키기 위해 다른 사람들을 도울 수도 있다.
사람들은 우리가 하는 일을 스스로 만들고 싶다는 욕망에 사로잡힌다.
그래서 그들은 결국 자신의 이익을 지키기 위해,

[프롬프트 3-1] 한국 영화 역사에서 가장 인상 깊었던 장면은
=> 한국 영화 역사에서 가장 인상 깊었던 장면은 바로 '장면'이다.
19세기 전반까지만 해도 흑백으로만 이뤄졌던 흑백 영화의 시대는 20세기에 접어들면서 다시 하나의 물결이 일기 시작했다.
영화가 가진 힘과 영향력은 21세기까지 지속된다.
그렇게 되면 전 세계 관객은 영화나 TV 드라마, 다큐멘터리 등을 통해 보다 쉽게 자신의 정체성을 확인할 수 있게 된다.
현재
